# Data Engineering - Superstore Dataset

* Michael Pheasant

## Exploratory Data Analysis

In [1]:
# requires: Python3
# Preamble - Utility funcs
def dump_uniq_data(df, uniq_key, extra_cols, file):
    '''Dump data to CSV (UTF-8) ensuring only one row per uniq_key (take first vals of extra_cols). Returns count of rows.'''
    cols = [uniq_key] + extra_cols
    data = df.groupby(uniq_key)[cols].first()
    data.to_csv(file, encoding='utf-8', index=False)
    return data[uniq_key].count()

def group_filter_sort(df, key, extra_cols, filt_fn):
    '''Return sorted rows of unique values of [key, extra_cols] after grouping by key and applying filter. '''
    return df[[key] + extra_cols].drop_duplicates().groupby([key]).filter(filt_fn).sort_values(by=key)

In [2]:
# Import data and browse
import zipfile
import pandas as pd
import numpy as np
import decimal
import datetime as dt

pd.set_option('display.max_columns', 500)

# Re-read and inspect data applying types and conversions
cat_cols = ['Ship Mode', 'Segment', 'Country',
            'Market', 'Region',
            'Category', 'Sub-Category', 
            'Order Priority']
int_cols = ['Row ID', 'Quantity']
dec_cols = ['Sales', 'Discount', 'Profit', 'Shipping Cost']
dt_cols = ['Order Date', 'Ship Date']
id_cols = ['Row ID', 'Order ID', 'Customer ID', 'Product ID']

types = {**{k: np.int_ for k in int_cols}, 
         **{k:"category" for k in cat_cols}}
conv = {k: decimal.Decimal for k in dec_cols}
zf = zipfile.ZipFile('superstore-data.zip')

df = pd.read_csv(zf.open('superstore_dataset2011-2015.csv'),
                 parse_dates=dt_cols, dtype=types,
                 converters=conv,
                 sep=',', encoding='Windows-1252')

# Inspect sample data
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Postal Code,Market,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,42433,AG-2011-2040,2011-01-01,2011-06-01,Standard Class,TB-11280,Toby Braunhardt,Consumer,Constantine,Constantine,Algeria,NaN,Africa,Africa,OFF-TEN-10000025,Office Supplies,Storage,"Tenex Lockers, Blue",408.3,2,0,106.14,35.46,Medium
1,22253,IN-2011-47883,2011-01-01,2011-08-01,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,Australia,NaN,APAC,Oceania,OFF-SU-10000618,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.366,3,0.1,36.036,9.72,Medium
2,48883,HU-2011-1220,2011-01-01,2011-05-01,Second Class,AT-735,Annie Thurman,Consumer,Budapest,Budapest,Hungary,NaN,EMEA,EMEA,OFF-TEN-10001585,Office Supplies,Storage,"Tenex Box, Single Width",66.12,4,0,29.64,8.17,High
3,11731,IT-2011-3647632,2011-01-01,2011-05-01,Second Class,EM-14140,Eugene Moren,Home Office,Stockholm,Stockholm,Sweden,NaN,EU,North,OFF-PA-10001492,Office Supplies,Paper,"Enermax Note Cards, Premium",44.865,3,0.5,-26.055,4.82,High
4,22255,IN-2011-47883,2011-01-01,2011-08-01,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,Australia,NaN,APAC,Oceania,FUR-FU-10003447,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",113.67,5,0.1,37.77,4.7,Medium


### Basic Sanity Checks

In [3]:
# Check col types - note Postal Code has missing values

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Row ID          51290 non-null  int64         
 1   Order ID        51290 non-null  object        
 2   Order Date      51290 non-null  datetime64[ns]
 3   Ship Date       51290 non-null  datetime64[ns]
 4   Ship Mode       51290 non-null  category      
 5   Customer ID     51290 non-null  object        
 6   Customer Name   51290 non-null  object        
 7   Segment         51290 non-null  category      
 8   City            51290 non-null  object        
 9   State           51290 non-null  object        
 10  Country         51290 non-null  category      
 11  Postal Code     9994 non-null   float64       
 12  Market          51290 non-null  category      
 13  Region          51290 non-null  category      
 14  Product ID      51290 non-null  object        
 15  Ca

In [4]:
# Check min/max dates make sense
# - 2011-2015 seems reasonable

[(col, df[col].min(), df[col].max()) for col in dt_cols]

[('Order Date',
  Timestamp('2011-01-01 00:00:00'),
  Timestamp('2014-12-31 00:00:00')),
 ('Ship Date',
  Timestamp('2011-01-02 00:00:00'),
  Timestamp('2015-07-01 00:00:00'))]

In [5]:
# Check categories 
# - the count of unique values seems to make sense for each category 

[(col, df[col].describe()) for col in cat_cols]

[('Ship Mode',
  count              51290
  unique                 4
  top       Standard Class
  freq               30775
  Name: Ship Mode, dtype: object),
 ('Segment',
  count        51290
  unique           3
  top       Consumer
  freq         26518
  Name: Segment, dtype: object),
 ('Country',
  count             51290
  unique              147
  top       United States
  freq               9994
  Name: Country, dtype: object),
 ('Market',
  count     51290
  unique        7
  top        APAC
  freq      11002
  Name: Market, dtype: object),
 ('Region',
  count       51290
  unique         13
  top       Central
  freq        11117
  Name: Region, dtype: object),
 ('Category',
  count               51290
  unique                  3
  top       Office Supplies
  freq                31273
  Name: Category, dtype: object),
 ('Sub-Category',
  count       51290
  unique         17
  top       Binders
  freq         6152
  Name: Sub-Category, dtype: object),
 ('Order Priority',
  coun

In [6]:
# Check ID columns for 51290 facts
# - RowID is unique integer from 1..51290
# - 25035 unique order IDs
# - 1590 unique customer IDs
# - 10292 unique product IDs

[(col, df[col].describe()) for col in id_cols]

[('Row ID',
  count    51290.00000
  mean     25645.50000
  std      14806.29199
  min          1.00000
  25%      12823.25000
  50%      25645.50000
  75%      38467.75000
  max      51290.00000
  Name: Row ID, dtype: float64),
 ('Order ID',
  count              51290
  unique             25035
  top       CA-2014-100111
  freq                  14
  Name: Order ID, dtype: object),
 ('Customer ID',
  count        51290
  unique        1590
  top       PO-18850
  freq            97
  Name: Customer ID, dtype: object),
 ('Product ID',
  count               51290
  unique              10292
  top       OFF-AR-10003651
  freq                   35
  Name: Product ID, dtype: object)]

In [7]:
# Inspect rows for a single order
# - rows appear to be product line items for a multiple-product order

df.loc[df['Order ID'] == 'IN-2011-47883']

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Postal Code,Market,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
1,22253,IN-2011-47883,2011-01-01,2011-08-01,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,Australia,NaN,APAC,Oceania,OFF-SU-10000618,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.366,3,0.1,36.036,9.72,Medium
4,22255,IN-2011-47883,2011-01-01,2011-08-01,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,Australia,NaN,APAC,Oceania,FUR-FU-10003447,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",113.67,5,0.1,37.77,4.7,Medium
5,22254,IN-2011-47883,2011-01-01,2011-08-01,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,Australia,NaN,APAC,Oceania,OFF-PA-10001968,Office Supplies,Paper,"Eaton Computer Printout Paper, 8.5 x 11",55.242,2,0.1,15.342,1.8,Medium


### Order ID Analysis

In [8]:
# Are order IDs unique for a given customer and date?
# - NO! Different customers on different days can have the same Order ID

group_filter_sort(df, 'Order ID', ['Order Date', 'Customer ID'], lambda x: len(x) > 1).head(4)

,Order ID,Order Date,Customer ID
6679,AG-2012-2220,2012-09-11,MV-8190
43686,AG-2012-2220,2012-12-26,PO-8865
3797,CA-2012-9840,2012-01-11,RR-9525
40864,CA-2012-9840,2012-06-25,LB-6735


In [9]:
# Example of different customers with different products on different days 
# in different markets with the same Order ID

df.loc[df['Order ID'] == 'US-2011-130379']

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Postal Code,Market,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
37368,6867,US-2011-130379,2011-05-23,2011-05-28,Standard Class,SG-20890,Susan Gilcrest,Corporate,Buenos Aires,Buenos Aires,Argentina,NaN,LATAM,South,OFF-AR-10001850,Office Supplies,Art,"Boston Sketch Pad, Water Color",289.968,14,0.4,-164.472,22.11,Medium
37378,6868,US-2011-130379,2011-05-23,2011-05-28,Standard Class,SG-20890,Susan Gilcrest,Corporate,Buenos Aires,Buenos Aires,Argentina,NaN,LATAM,South,TEC-MA-10000122,Technology,Machines,"Okidata Calculator, Red",39.552,2,0.4,-11.888,1.52,Medium
40756,33115,US-2011-130379,2011-05-25,2011-05-29,Standard Class,JL-15235,Janet Lee,Consumer,Chicago,Illinois,United States,60623.0,US,Central,OFF-AP-10001394,Office Supplies,Appliances,Harmony Air Purifier,75.6,2,0.8,-166.32,8.27,Medium
40760,33116,US-2011-130379,2011-05-25,2011-05-29,Standard Class,JL-15235,Janet Lee,Consumer,Chicago,Illinois,United States,60623.0,US,Central,FUR-FU-10002553,Furniture,Furnishings,"Electrix Incandescent Magnifying Lamp, Black",29.32,2,0.6,-24.189,2.45,Medium


### Sale Date Dimension

In [10]:
# - Create a composite key SaleDateID for customer sales on a certain date 
#   based on customer and date

df['SaleDateID'] = df['Order Date'].apply(dt.date.isoformat) + '~' + df['Order ID'] + '~' + df['Customer ID']

print ([(col, df[col].describe()) for col in  ['SaleDateID']])

df.loc[df['Order ID'] == 'US-2011-130379']


[('SaleDateID', count                                  51290
unique                                 25754
top       2014-09-21~CA-2014-100111~SV-20365
freq                                      14
Name: SaleDateID, dtype: object)]


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Postal Code,Market,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,SaleDateID
37368,6867,US-2011-130379,2011-05-23,2011-05-28,Standard Class,SG-20890,Susan Gilcrest,Corporate,Buenos Aires,Buenos Aires,Argentina,NaN,LATAM,South,OFF-AR-10001850,Office Supplies,Art,"Boston Sketch Pad, Water Color",289.968,14,0.4,-164.472,22.11,Medium,2011-05-23~US-2011-130379~SG-20890
37378,6868,US-2011-130379,2011-05-23,2011-05-28,Standard Class,SG-20890,Susan Gilcrest,Corporate,Buenos Aires,Buenos Aires,Argentina,NaN,LATAM,South,TEC-MA-10000122,Technology,Machines,"Okidata Calculator, Red",39.552,2,0.4,-11.888,1.52,Medium,2011-05-23~US-2011-130379~SG-20890
40756,33115,US-2011-130379,2011-05-25,2011-05-29,Standard Class,JL-15235,Janet Lee,Consumer,Chicago,Illinois,United States,60623.0,US,Central,OFF-AP-10001394,Office Supplies,Appliances,Harmony Air Purifier,75.6,2,0.8,-166.32,8.27,Medium,2011-05-25~US-2011-130379~JL-15235
40760,33116,US-2011-130379,2011-05-25,2011-05-29,Standard Class,JL-15235,Janet Lee,Consumer,Chicago,Illinois,United States,60623.0,US,Central,FUR-FU-10002553,Furniture,Furnishings,"Electrix Incandescent Magnifying Lamp, Black",29.32,2,0.6,-24.189,2.45,Medium,2011-05-25~US-2011-130379~JL-15235


In [11]:
# Add week, weekday and other derived dates

pd.Timestamp('1/1/1970').weekday()
df['SaleDate'] = df['Order Date']
df['SaleYear'] = df['Order Date'].apply(lambda x: x.year)
df['SaleMonth'] = df['Order Date'].apply(lambda x: x.month) # 1..12
df['SaleISOWeekNumber'] = df['Order Date'].apply(lambda x: x.isocalendar()[1]) # 1..53
df['SaleISOWeekDay'] = df['Order Date'].apply(lambda x: x.isocalendar()[2]) # 1..7
df.head()

#from dateutil.parser import parse
#parse('2020-02-09').year
#parse('2020-12-09').month

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Postal Code,Market,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,SaleDateID,SaleDate,SaleYear,SaleMonth,SaleISOWeekNumber,SaleISOWeekDay
0,42433,AG-2011-2040,2011-01-01,2011-06-01,Standard Class,TB-11280,Toby Braunhardt,Consumer,Constantine,Constantine,Algeria,NaN,Africa,Africa,OFF-TEN-10000025,Office Supplies,Storage,"Tenex Lockers, Blue",408.3,2,0,106.14,35.46,Medium,2011-01-01~AG-2011-2040~TB-11280,2011-01-01,2011,1,52,6
1,22253,IN-2011-47883,2011-01-01,2011-08-01,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,Australia,NaN,APAC,Oceania,OFF-SU-10000618,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.366,3,0.1,36.036,9.72,Medium,2011-01-01~IN-2011-47883~JH-15985,2011-01-01,2011,1,52,6
2,48883,HU-2011-1220,2011-01-01,2011-05-01,Second Class,AT-735,Annie Thurman,Consumer,Budapest,Budapest,Hungary,NaN,EMEA,EMEA,OFF-TEN-10001585,Office Supplies,Storage,"Tenex Box, Single Width",66.12,4,0,29.64,8.17,High,2011-01-01~HU-2011-1220~AT-735,2011-01-01,2011,1,52,6
3,11731,IT-2011-3647632,2011-01-01,2011-05-01,Second Class,EM-14140,Eugene Moren,Home Office,Stockholm,Stockholm,Sweden,NaN,EU,North,OFF-PA-10001492,Office Supplies,Paper,"Enermax Note Cards, Premium",44.865,3,0.5,-26.055,4.82,High,2011-01-01~IT-2011-3647632~EM-14140,2011-01-01,2011,1,52,6
4,22255,IN-2011-47883,2011-01-01,2011-08-01,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,Australia,NaN,APAC,Oceania,FUR-FU-10003447,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",113.67,5,0.1,37.77,4.7,Medium,2011-01-01~IN-2011-47883~JH-15985,2011-01-01,2011,1,52,6


In [12]:
# Dump SaleDate data - 25754 rows

dump_uniq_data(df, 'SaleDateID', 
               ['SaleDate', 'SaleYear', 'SaleMonth', 'SaleISOWeekNumber', 'SaleISOWeekDay'], 
               'load_saledate.csv')

25754

### Customer Dimension

In [13]:
# Does one Customer ID and Segment have multiple values
# - No

group_filter_sort(df, 'Customer ID', ['Customer Name', 'Segment'], lambda x: len(x) > 1).head(4)

,Customer ID,Customer Name,Segment


In [14]:
# Dump Customer data - 1590 rows

dump_uniq_data(df, 'Customer ID', ['Segment', 'Customer Name'], 'load_customer.csv')

1590

### Product Dimension

In [15]:
# Is Product ID unique for Product Name
# - No: 457 / 10292 (4.4%) total unique IDs have 836 different names
# - For now, take first Product Name found
# - later investigate if any conflicting names are high-value and would affect analysis

key = 'Product ID'
uniq_ids = df[[key, 'Product Name']].drop_duplicates().groupby([key]).filter(lambda x: len(x) == 1).sort_values(by=key)[key].count()
print('Unique IDs: ', uniq_ids)

dup_prods = df[[key, 'Product Name']].drop_duplicates().groupby([key]).filter(lambda x: len(x) > 1).sort_values(by=key)
dup_ids = dup_prods['Product ID'].drop_duplicates().count()
print('Dup IDs: ', dup_ids)
print('Uniq + Dup IDs: ', uniq_ids + dup_ids)
print('Dup: Names: ', dup_prods['Product Name'].drop_duplicates().count())

dup_prods.head(4)

Unique IDs:  9835
Dup IDs:  457
Uniq + Dup IDs:  10292
Dup: Names:  836


,Product ID,Product Name
4684,FUR-BO-10000087,"Dania Classic Bookcase, Mobile"
4848,FUR-BO-10000087,"Sauder Corner Shelving, Pine"
6102,FUR-BO-10000112,"Dania Corner Shelving, Pine"
15786,FUR-BO-10000112,"Bush Birmingham Collection Bookcase, Dark Cherry"


In [16]:
# Do any Product IDs have conflicting Category/Sub-Category?
# - Yes! but only one product has a conflicting sub-category 
#   so no significant impact on results, can take either value

group_filter_sort(df, 'Product ID', ['Category', 'Sub-Category'], lambda x: len(x) > 1)

,Product ID,Category,Sub-Category
4390,OFF-AVE-10002102,Office Supplies,Binders
13442,OFF-AVE-10002102,Office Supplies,Labels


In [17]:
# Dump product data - 10292 rows

dump_uniq_data(df, 'Product ID', ['Category', 'Sub-Category', 'Product Name'], 'load_product.csv')

10292

### Market Region Dimension

In [18]:
# A customer orders in multiple Markets

group_filter_sort(df, 'Customer ID', ['Segment', 'Market', 'Customer Name'], lambda x: len(x) >1).head()

,Customer ID,Segment,Market,Customer Name
1110,AA-10315,Consumer,APAC,Alex Avila
4834,AA-10315,Consumer,US,Alex Avila
9533,AA-10315,Consumer,LATAM,Alex Avila
10735,AA-10315,Consumer,EU,Alex Avila
4242,AA-10375,Consumer,US,Allen Armold


In [19]:
# A product is sold in multiple markets

group_filter_sort(df, 'Product ID', ['Market', 'Product Name'], lambda x: len(x) >1).head()

,Product ID,Market,Product Name
17240,FUR-ADV-10000002,Africa,"Advantus Photo Frame, Duo Pack"
17474,FUR-ADV-10000002,EMEA,"Advantus Photo Frame, Duo Pack"
7404,FUR-ADV-10000183,EMEA,"Advantus Photo Frame, Black"
34679,FUR-ADV-10000183,Africa,"Advantus Photo Frame, Black"
18238,FUR-ADV-10000188,Africa,"Advantus Stacking Tray, Erganomic"


In [20]:
# An order ID is in multiple markets
# - but this is because different customers on different days in different markets can get the same Order ID

group_filter_sort(df, 'Order ID', ['Market'], lambda x: len(x) >1).head()
#group_filter_sort(df, 'Order ID', ['Customer ID', 'Market'], lambda x: len(x) >1).head()
#df.loc[df['Order ID'] == 'US-2011-130379']

,Order ID,Market
37368,US-2011-130379,LATAM
40756,US-2011-130379,US
34034,US-2011-133130,LATAM
44903,US-2011-133130,US
41705,US-2011-144078,US


In [21]:
# Add Market Region ID as composite of market & region to link direct to Sale Item fact
# - may be better to use surrogate key but this is easier right now

df['MarketRegionID'] = df['Market'].astype('string') + '~' + df['Region'].astype('string')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Postal Code,Market,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,SaleDateID,SaleDate,SaleYear,SaleMonth,SaleISOWeekNumber,SaleISOWeekDay,MarketRegionID
0,42433,AG-2011-2040,2011-01-01,2011-06-01,Standard Class,TB-11280,Toby Braunhardt,Consumer,Constantine,Constantine,Algeria,NaN,Africa,Africa,OFF-TEN-10000025,Office Supplies,Storage,"Tenex Lockers, Blue",408.3,2,0,106.14,35.46,Medium,2011-01-01~AG-2011-2040~TB-11280,2011-01-01,2011,1,52,6,Africa~Africa
1,22253,IN-2011-47883,2011-01-01,2011-08-01,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,Australia,NaN,APAC,Oceania,OFF-SU-10000618,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.366,3,0.1,36.036,9.72,Medium,2011-01-01~IN-2011-47883~JH-15985,2011-01-01,2011,1,52,6,APAC~Oceania
2,48883,HU-2011-1220,2011-01-01,2011-05-01,Second Class,AT-735,Annie Thurman,Consumer,Budapest,Budapest,Hungary,NaN,EMEA,EMEA,OFF-TEN-10001585,Office Supplies,Storage,"Tenex Box, Single Width",66.12,4,0,29.64,8.17,High,2011-01-01~HU-2011-1220~AT-735,2011-01-01,2011,1,52,6,EMEA~EMEA
3,11731,IT-2011-3647632,2011-01-01,2011-05-01,Second Class,EM-14140,Eugene Moren,Home Office,Stockholm,Stockholm,Sweden,NaN,EU,North,OFF-PA-10001492,Office Supplies,Paper,"Enermax Note Cards, Premium",44.865,3,0.5,-26.055,4.82,High,2011-01-01~IT-2011-3647632~EM-14140,2011-01-01,2011,1,52,6,EU~North
4,22255,IN-2011-47883,2011-01-01,2011-08-01,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,Australia,NaN,APAC,Oceania,FUR-FU-10003447,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",113.67,5,0.1,37.77,4.7,Medium,2011-01-01~IN-2011-47883~JH-15985,2011-01-01,2011,1,52,6,APAC~Oceania


In [22]:
# Dump MarketRegion data - 18 rows

dump_uniq_data(df, 'MarketRegionID', ['Market', 'Region'], 'load_marketregion.csv')

18

### Sale Item Facts

In [23]:
# Dump data for SaleItem table

df[['Row ID', 'Customer ID', 'Product ID', 'SaleDateID', 'MarketRegionID',
   'Sales', 'Quantity', 'Discount', 'Profit', 'Shipping Cost']].to_csv('load_saleitem.csv', encoding='utf-8', index=False)